In [1]:
#%pip install lxml
import lxml
import numpy as np
import pandas as pd
import cv2
import xml.etree.ElementTree as et
import os
import matplotlib.pyplot as pt

In [ ]:
#read xml file

anno="/kaggle/input/pascal-voc-2012/VOC2012/Annotations/"
def read_xml_file(data):
    tree=et.parse(data)
    root=tree.getroot()
    image_name=root.find('filename').text
    
    image_width=int(root.find("size").find("width").text)
    image_height=int(root.find("size").find("height").text)
  #  image_depth=root.find("size").find("depth").text
    new_width=224/image_width
    new_heigh=224/image_height
  #  image_size=(int(image_width),int(image_height))
    
    b_box=[]
    names=[]
    for i in root.findall("object"):
        name=i.find("name").text
        
        names.append(name)
        x_min=float(i.find("bndbox").find("xmin").text)*new_width
        y_min=float(i.find("bndbox").find("ymin").text)*new_heigh
        x_max=float(i.find("bndbox").find("xmax").text)*new_width
        y_max=float(i.find("bndbox").find("ymax").text)*new_heigh
        b_box.append((x_min,y_min,x_max,y_max))
    
    return(image_name,names,b_box)
    
dict_bbox={}

for i in os.listdir(anno):
    image_name,name,b_box=read_xml_file(anno+i)
    dict_bbox[image_name]=(name,b_box)
    


In [ ]:
#read all images and label bounding box 
def read_image(image):
    im=cv2.imread(image)
    im=cv2.resize(im,(224,224))
    return im

train=[]
label=[]
image="/kaggle/input/pascal-voc-2012/VOC2012/JPEGImages/"
images=[]
for i in os.listdir(image):
    train.append(read_image(image+i))
    label.append(dict_bbox[i])

In [ ]:
#build vgg16 as backbone for faster rcnn
import tensorflow as tf
from keras.layers import Conv2D,MaxPool2D,Dense,Flatten
from keras.models import Sequential

model=Sequential()

model.add(Conv2D(kernel_size=(3,3),filters=64,input_shape=(224,224,3),activation='relu',padding='same',name='block1_conv1'))
model.add(Conv2D(kernel_size=(3,3),filters=64,activation='relu',padding='same',name='block1_conv2'))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2),name= 'block1_pool'))

model.add(Conv2D(filters=128,kernel_size=(3,3),activation='relu',padding='same',name= 'block2_conv1'))
model.add(Conv2D(filters=128,kernel_size=(3,3),activation='relu',padding='same',name= 'block2_conv2',))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2),name='block2_pool'))

model.add(Conv2D(kernel_size=(3,3),filters=256,activation='relu',padding='same',name='block3_conv1'))
model.add(Conv2D(kernel_size=(3,3),filters=256,activation='relu',padding='same',name='block3_conv2'))
model.add(Conv2D(kernel_size=(3,3),filters=256,activation='relu',padding='same',name='block3_conv3'))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2),name='block3_pool'))

model.add(Conv2D(kernel_size=(3,3),filters=512,activation='relu',padding='same',name='block4_conv1'))
model.add(Conv2D(kernel_size=(3,3),filters=512,activation='relu',padding='same',name='block4_conv2'))
model.add(Conv2D(kernel_size=(3,3),filters=512,activation='relu',padding='same',name='block4_conv3'))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2),name='block4_pool'))

model.add(Conv2D(kernel_size=(3,3),filters=512,activation='relu',padding='same',name='block5_conv1'))
model.add(Conv2D(kernel_size=(3,3),filters=512,activation='relu',padding='same',name='block5_conv2'))
model.add(Conv2D(kernel_size=(3,3),filters=512,activation='relu',padding='same',name='block5_conv3'))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2),name='block5_pool'))

path="/kaggle/input/vgg16/vgg16_weights_tf_dim_ordering_tf_kernels.h5"

model.load_weights(path,by_name=True)
model.compile(loss='cross_entropy',optimizer="adam")
model.summary()

OUT_VGG16=model.outputs

print(OUT_VGG16)



In [ ]:
#build RPN to faster rcnn
from tensorflow.keras.models import Model

def rpn_model(num_anchor,model_use):
    
    conv=Conv2D(kernel_size=(3,3),filters=512,activation='relu',padding='same',strides=(1,1))(model_use)
    
    rpn_cls=Conv2D(filters=num_anchor*2,kernel_size=(1,1),activation='softmax')(conv)
    
    rpn_reg=Conv2D(filters=num_anchor*4,kernel_size=(1,1))(conv)
    
    #return Model(inputs=model_use,outputs=[anch,anc_reg])
    return rpn_cls,rpn_reg


rpn_cls, rpn_reg=rpn_model(9,OUT_VGG16[0])
#print(rbn.summary())

In [ ]:
#create ROIS File for Faster RCNN

def create_roi(bound_box,ima_num,batch):
    #batch_index=0
    batch_index=ima_num//batch
    roi=[]
    for i in bound_box:
        x1=i[0]/224
        y1=i[1]/224
        x2=i[2]/224
        y2=i[3]/224
        roi.append(np.array([batch_index,y1,x1,y2,x2]))
        
    return roi

#print("=============================================================")
roi_arr=[]
count=0
for i in(label):
    
    roi_arr.append(create_roi(i[1],count,50))

    count+=1
#print(label[0][1])
print(roi_arr[0])
    

In [ ]:
#Create roi align layer 

import tensorflow as tf

from tensorflow.keras.layers import Lambda

def roi_align(feature_map,rois,out_size=(7,7)):
    
    boxs=tf.convert_to_tensor(rois,dtype=tf.float32)
    
    crob=tf.image.crob_and_resize(feature_map,boxs,box_indices=boxs[:,0],crop_size=out_size)
    
    return crob

In [10]:
#here we concatenate every thing 
from tensorflow.keras.layers import Input,Lambda

#input_image=Input(shape=(None,None,3))

feture=OUT_VGG16[0]

rois=roi_arr

rpn_cls, rpn_reg=rpn_cls, rpn_reg

align_rois=Lambda(lambda x:roi_align(x[0],x[1],x[2]),output_shape=(None,7,7,512))([feture,rois])

model=Model(inputs=input_image,outputs=[rpn_cls, rpn_reg,align_rois])
model.compile(optimizer='adam', loss={'classification': 'categorical_crossentropy', 'regression': 'mean_squared_error', 'output': 'categorical_crossentropy'})

#model.summary()

NameError: name 'OUT_VGG16' is not defined

In [1]:
#this to edit h5 file we donit use it in the code we just trye
import h5py
import shutil

data_h5="/kaggle/input/vgg16/vgg16_weights_tf_dim_ordering_tf_kernels.h5"
#data_h5_work="/kaggle/working/vgg15.h5"

#shutil.copy(data_h5, data_h5_work)

file=h5py.File(data_h5)
print(list(file.keys()))
#with h5py.File(data_h5,'r+') as f:
    """if 'fc1' in f:
        del(f['fc1'])
    if 'fc2' in f:
        del(f['fc2'])
    if 'flatten' in f:
        del(f['flatten'])
    if 'predictions' in f:
        del(f['predictions'])"""
    
    #print(list(f.keys()))
    
    
#file=h5py.File(data_h5_work)
#print(list(file.keys()))



IndentationError: unexpected indent (2416601833.py, line 13)

In [9]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Input, Conv2D, Flatten, Dense, Lambda, Reshape
from tensorflow.keras.models import Model
import tensorflow as tf

# Define VGG16 as feature extractor
#def build_feature_extractor(input_shape):
base_model = VGG16(include_top=False, input_shape=input_shape)
print(base_model.outputs)
base_model.summary()
   # x = base_model.output
#    return Model(inputs=base_model.input, outputs=x)

#input_shape = (None, None, 3)  # Image shape
#feature_extractor = build_feature_extractor(input_shape)


[<KerasTensor shape=(None, None, None, 512), dtype=float32, sparse=False, name=keras_tensor_132>]


Model: "vgg16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_6 (InputLayer)      │ (None, None, None, 3)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, None, None, 64) │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, None, None, 64) │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, None, None, 64) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, None, None,     │        73,856 │
│                                 │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, None, None,     │       147,584 │
│                                 │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, None, None,     │             0 │
│                                 │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, None, None,     │       295,168 │
│                                 │ 256)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, None, None,     │       590,080 │
│                                 │ 256)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, None, None,     │       590,080 │
│                                 │ 256)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, None, None,     │             0 │
│                                 │ 256)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, None, None,     │     1,180,160 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, None, None,     │             0 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │             

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 14,714,688 (56.13 MB)

 Non-trainable params: 0 (0.00 B)